<a href="https://colab.research.google.com/github/matiasfeliu92/torneos_primera_division_arg/blob/master/ETL/ETL_Torneos_Primera_Division.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gspread==3.6
# !pip list

  Attempting uninstall: gspread
    Found existing installation: gspread 3.4.2
    Uninstalling gspread-3.4.2:
      Successfully uninstalled gspread-3.4.2


In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import datetime
import os
from google.colab import drive
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
print(gspread.__version__)

3.6.0


In [3]:
drive.mount('/content/drive')
drive_root = '/content/drive/My Drive/google_sheets_credentials'
files = os.listdir(drive_root)
print(files)

Mounted at /content/drive
['client_secret_2_3384657644-o03i4l3u9vddiv6n1h51qg6i0h8s1omf.apps.googleusercontent.com.json', 'centered-flow-368315-68e92ed40d61.json']


In [4]:
creden = '/content/drive/My Drive/google_sheets_credentials/centered-flow-368315-68e92ed40d61.json'
gc = gspread.service_account(filename=creden)
url_sheet = "https://docs.google.com/spreadsheets/d/16Ucu31Cg9_v_SAUgBc5dDMTkxJNacC6d0cTi7Rwa6sg/edit#gid=1024039146"
sh = gc.open_by_url(url_sheet)

In [5]:
Torneos = [
    {
        'nombre': "Liga Profesional Argentina 2017",
        'url': "https://www.resultados-futbol.com/primera_division_argentina2017/grupo1/calendario",
    },
    {
        'nombre': "Liga Profesional Argentina 2018",
        'url': "https://www.resultados-futbol.com/primera_division_argentina2018/grupo1/calendario"
    },
    {
        'nombre': "Liga Profesional Argentina 2019",
        'url': "https://www.resultados-futbol.com/primera_division_argentina2019/grupo1/calendario"
    },
    {
        'nombre': "Liga Profesional Argentina 2020",
        'url': "https://www.resultados-futbol.com/primera_division_argentina2020/grupo1/calendario"
    },
    {
        'nombre': "Liga Profesional Argentina 2021",
        'url': "https://www.resultados-futbol.com/primera_division_argentina2021/grupo1/calendario"
    },
    {
        'nombre': "Liga Profesional Argentina 2022",
        'url': "https://www.resultados-futbol.com/primera_division_argentina2022/grupo1/calendario"
    },
    {
        'nombre': "Liga Profesional Argentina 2023",
        'url': "https://www.resultados-futbol.com/primera_division_argentina2023/grupo1/calendario"
    },
    {
        'nombre': "Copa Liga Profesional Argentina 2023",
        'url': "https://www.resultados-futbol.com/copa_liga_profesional_argentina2023/grupo1/calendario"
    }
]

In [6]:
df_url_torneos = pd.DataFrame(Torneos)
df_url_torneos

worksheet_name = "URLS_Fuente"
worksheet = sh.worksheet(worksheet_name)
set_with_dataframe(worksheet, df_url_torneos)

In [7]:
def get_tournament_results(url, torneo):
  page = requests.get(url)
  content = page.content
  soup = BeautifulSoup(content, "html.parser")
  jornadas = soup.find_all(id='col-resultados')
  results = []
  for jornada in jornadas:
      nro_jornada = jornada.find(class_='titlebox').text
      tablaJornada = jornada.find('table')
      partidosJornada = tablaJornada.find_all('tr')
      for partido in partidosJornada:
          data = partido.find_all('td')
          if len(data) > 0:
              fecha_partido = data[0].text.strip()
              equipo_local = data[1].find('img').get('alt').strip()
              resultado = data[2].find('a').text.strip()
              equipo_visitante = data[3].find('img').get('alt').strip()
              try:
                  equipo_local_resultado = resultado.split('-')[0].strip()
                  equipo_visitante_resultado = resultado.split('-')[1].strip()
              except:
                  equipo_local_resultado = '-'
                  equipo_visitante_resultado = '-'
              results.append({
                  'nombre_torneo': torneo,
                  'nro_jornada': nro_jornada,
                  'fecha_partido': fecha_partido,
                  'equipo_local': equipo_local,
                  'resultado': resultado,
                  'equipo_visitante': equipo_visitante,
                  'equipo_local_resultado': equipo_local_resultado,
                  'equipo_visitante_resultado': equipo_visitante_resultado
              })
  return results

In [8]:
def get_best_players(url, torneo):
  page = requests.get(url)
  content = page.content
  soup = BeautifulSoup(content, "html.parser")
  goleadores = soup.find_all(id='informacion-goleadores')
  best_players = []
  for goleador in goleadores:
    jug_goleador = goleador.find('table')
    jug_goleador_data = jug_goleador.find_all('tr')
    for jugador in jug_goleador_data:
      data = jugador.find_all('td')
      if len(data) > 0:
        nombre = data[0].text.strip()
        goles = data[1].text.strip()
        equipo = data[2].text.strip()
        if nombre != 'Nombre' and goles != 'Goles' and equipo != 'Equipo':
          best_players.append({
              'nombre_torneo': torneo,
              'nombre': nombre,
              'cant_goles': goles,
              'equipo': equipo,
          })
  return best_players

In [9]:
def get_positions_table(url, nombre):
  page = requests.get(url)
  content = page.content
  soup = BeautifulSoup(content, "html.parser")
  table_positions = []
  positions = soup.find_all(id='clasificacion')
  for position in positions:
    position_table = position.find('table')
    position_table_data = position_table.find_all('tr')
    for pos in position_table_data:
      data = pos.find_all('td')
      if len(data) > 0:
        equipo = data[0].text.strip()
        pj = data[1].text.strip()
        gf = data[2].text.strip()
        gc = data[3].text.strip()
        puntos = data[4].text.strip()
        if equipo != 'Equipo' and pj != 'PJ' and gf != 'GF' and gc != 'GC' and puntos != 'Puntos':
          table_positions.append({
              'nombre_torneo': torneo,
              'equipo': equipo,
              'PJ': pj,
              'GF': gf,
              'GC': gc,
              'puntos': puntos
          })
  return table_positions

In [10]:
df_tournament_results = pd.DataFrame()
df_best_players = pd.DataFrame()
df_table_positions = pd.DataFrame()

for index, row in df_url_torneos.iterrows():
  url = row['url']
  torneo = row['nombre']

  df = pd.DataFrame(get_tournament_results(url, torneo))
  df_tournament_results = pd.concat([df_tournament_results, df], ignore_index=True)

  df = pd.DataFrame(get_best_players(url, torneo))
  df_best_players = pd.concat([df_best_players, df], ignore_index=True)

  df = pd.DataFrame(get_positions_table(url, torneo))
  df_table_positions = pd.concat([df_table_positions, df], ignore_index=True)

In [11]:
worksheet_name = "tournament_results"
worksheet = sh.worksheet(worksheet_name)
worksheet.clear()
set_with_dataframe(worksheet, df_tournament_results)

df_tournament_results

,nombre_torneo,nro_jornada,fecha_partido,equipo_local,resultado,equipo_visitante,equipo_local_resultado,equipo_visitante_resultado
0,Liga Profesional Argentina 2017,Jornada 1,30 Ago 16,Aldosivi,0 - 2,Colón,0,2
1,Liga Profesional Argentina 2017,Jornada 1,28 Ago 16,River Plate,4 - 1,Banfield,4,1
2,Liga Profesional Argentina 2017,Jornada 1,27 Ago 16,Racing Club,1 - 1,Talleres Córdoba,1,1
3,Liga Profesional Argentina 2017,Jornada 1,28 Ago 16,San Lorenzo,2 - 2,San Martín San Juan,2,2
4,Liga Profesional Argentina 2017,Jornada 1,27 Ago 16,Rosario Central,0 - 0,Defensa y Justicia,0,0
...,...,...,...,...,...,...,...,...
2701,Copa Liga Profesional Argentina 2023,Jornada 14,26 Nov 23,San Lorenzo,x - x,Central Córdoba,x,x
2702,Copa Liga Profesional Argentina 2023,Jornada 14,26 Nov 23,Newell's Old Boys,x - x,Defensa y Justicia,x,x
2703,Copa Liga Profesional Argentina 2023,Jornada 14,26 Nov 23,Godoy Cruz,x - x,Boca Juniors,x,x
2704,Copa Liga Profesional Argentina 2023,Jornada 14,26 Nov 23,River Plate,x - x,Instituto,x,x


In [12]:
worksheet_name = "best_players"
worksheet = sh.worksheet(worksheet_name)
worksheet.clear()
set_with_dataframe(worksheet, df_best_players)

df_best_players

,nombre_torneo,nombre,cant_goles,equipo
0,Liga Profesional Argentina 2017,D. Benedetto,21,BOC
1,Liga Profesional Argentina 2017,S. Driussi,17,RIV
2,Liga Profesional Argentina 2017,J. Sand,15,LAN
3,Liga Profesional Argentina 2017,M. Pavone,13,VEL
4,Liga Profesional Argentina 2017,I. Scocco,12,NOB
...,...,...,...,...
75,Copa Liga Profesional Argentina 2023,A. Martínez,5,INS
76,Copa Liga Profesional Argentina 2023,S. Rondón,5,RIV
77,Copa Liga Profesional Argentina 2023,S. Castro,5,VEL
78,Copa Liga Profesional Argentina 2023,Tomás Galván,4,COL


In [13]:
worksheet_name = "table_positions"
worksheet = sh.worksheet(worksheet_name)
worksheet.clear()
set_with_dataframe(worksheet, df_table_positions)

df_table_positions

,nombre_torneo,equipo,PJ,GF,GC,puntos
0,Liga Profesional Argentina 2017,Boca Juniors,30,62,25,63
1,Liga Profesional Argentina 2017,River Plate,30,51,28,56
2,Liga Profesional Argentina 2017,Estudiantes,30,46,26,56
3,Liga Profesional Argentina 2017,Racing Club,30,51,40,55
4,Liga Profesional Argentina 2017,Banfield,30,42,35,54
...,...,...,...,...,...,...
213,Copa Liga Profesional Argentina 2023,Estudiantes,12,9,12,13
214,Copa Liga Profesional Argentina 2023,Boca Juniors,11,13,14,11
215,Copa Liga Profesional Argentina 2023,Lanús,12,8,11,11
216,Copa Liga Profesional Argentina 2023,San Lorenzo,11,7,10,11
